In [30]:
import pandas as pd

In [31]:
# Load the pollution_dataset
traffic_data = pd.read_csv("D:/600/traffic project/traffic-volume-survey.csv")

In [32]:
traffic_data.head()

,geo_point_2d,geo_shape,latitude,longitude,road_name,installed_year,range,type,date,easting,...,projection,postcode,pc85th_kmh,volume_12h,volume_24h,comm_vhcl,peakvol,suburb_name,ward_name,gisfid
0,"-38.1216300043, 145.2924700205","{""coordinates"": [145.2924700205, -38.121630004...",-38.121630,145.292470,ANNABELLA STREET,2022,Between 50km/h and 60km/h,W,2022-08-24,350326,...,MGA2020 zone 55,3977,50.5,546,658,3.15,10.27,CRANBOURNE EAST,BALLA BALLA,506
1,"-38.0790430044, 145.3301540239","{""coordinates"": [145.3301540239, -38.079043004...",-38.079043,145.330154,ALISMA BOULEVARD,2022,Between 50km/h and 60km/h,W,2022-08-24,353544,...,MGA2020 zone 55,3977,56.8,2790,3708,2.72,10.85,CRANBOURNE NORTH,SPRINGFIELD,514
2,"-38.0973230017, 145.2458930241","{""coordinates"": [145.2458930241, -38.097323001...",-38.097323,145.245893,QUARTERS BOULEVARD,2022,Less than 30km/h,W,2022-08-24,346192,...,MGA2020 zone 55,3977,27.0,368,402,3.24,16.79,CRANBOURNE WEST,MAYFIELD,517
3,"-37.9748000045, 145.2722920164","{""coordinates"": [145.2722920164, -37.974800004...",-37.974800,145.272292,HANNA DRIVE,2022,Between 50km/h and 60km/h,W,2022-08-24,348254,...,MGA2020 zone 55,3802,54.2,3661,4293,6.00,21.70,ENDEAVOUR HILLS,FOUR OAKS,520
4,"-38.0443360031, 145.2755090228","{""coordinates"": [145.2755090228, -38.044336003...",-38.044336,145.275509,CORAL DRIVE,2022,Between 50km/h and 60km/h,W,2022-08-24,348680,...,MGA2020 zone 55,3976,50.0,3244,4017,4.39,11.94,HAMPTON PARK,RIVER GUM,524


In [33]:
# Update the function to categorize 'Less than 30km/h' as 'low'
def categorize_traffic_level_updated(speed_range):
    if speed_range == 'Between 30km/h and 40km/h' or speed_range == 'Less than 30km/h':
        return 'low'
    elif speed_range == 'Between 40km/h and 50km/h':
        return 'moderate'
    elif speed_range == 'Between 50km/h and 60km/h':
        return 'high'
    elif speed_range == 'More than 60km/h':
        return 'extremely high'
    else:
        return 'undefined'  # Just in case there are other undefined categories

# Apply the updated categorization
traffic_data['traffic level'] = traffic_data['range'].apply(categorize_traffic_level_updated)
traffic_data[['range', 'traffic level']].head()


,range,traffic level
0,Between 50km/h and 60km/h,high
1,Between 50km/h and 60km/h,high
2,Less than 30km/h,low
3,Between 50km/h and 60km/h,high
4,Between 50km/h and 60km/h,high


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

#independent and dependent varaibles selection
X = traffic_data[['pc85th_kmh', 'volume_12h', 'volume_24h', 'comm_vhcl', 'peakvol']]
y = traffic_data['range']  # Target variable

#Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Save the optimized model
joblib.dump(model, 'optimized_model.pkl')

print(accuracy)


0.9949494949494949


In [35]:
print(report)

                           precision    recall  f1-score   support

Between 30km/h and 40km/h       0.95      1.00      0.98        21
Between 40km/h and 50km/h       1.00      1.00      1.00       124
Between 50km/h and 60km/h       1.00      0.99      1.00       171
         Less than 30km/h       1.00      0.90      0.95        10
         More than 60km/h       0.99      1.00      0.99        70

                 accuracy                           0.99       396
                macro avg       0.99      0.98      0.98       396
             weighted avg       1.00      0.99      0.99       396



In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Train a Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)
y_pred_logistic = logistic_model.predict(X_test)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
report_logistic = classification_report(y_test, y_pred_logistic)



# Train a Decision Tree model
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)
y_pred_tree = decision_tree_model.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
report_tree = classification_report(y_test, y_pred_tree)




c:\Users\yaswa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\yaswa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\yaswa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.p

In [37]:
#Saving the logisti model
joblib.dump(logistic_model, 'log_model.pkl')

['log_model.pkl']

In [38]:
#Saving the logisti model
joblib.dump(decision_tree_model, 'dec_model.pkl')

['dec_model.pkl']

In [39]:
print(accuracy_logistic)

0.43686868686868685


In [40]:
print(report_logistic)

                           precision    recall  f1-score   support

Between 30km/h and 40km/h       0.00      0.00      0.00        21
Between 40km/h and 50km/h       0.36      0.04      0.07       124
Between 50km/h and 60km/h       0.44      0.92      0.60       171
         Less than 30km/h       0.00      0.00      0.00        10
         More than 60km/h       0.39      0.16      0.22        70

                 accuracy                           0.44       396
                macro avg       0.24      0.22      0.18       396
             weighted avg       0.37      0.44      0.32       396



In [41]:
print(accuracy_tree)

1.0


In [42]:
print(report_tree)

                           precision    recall  f1-score   support

Between 30km/h and 40km/h       1.00      1.00      1.00        21
Between 40km/h and 50km/h       1.00      1.00      1.00       124
Between 50km/h and 60km/h       1.00      1.00      1.00       171
         Less than 30km/h       1.00      1.00      1.00        10
         More than 60km/h       1.00      1.00      1.00        70

                 accuracy                           1.00       396
                macro avg       1.00      1.00      1.00       396
             weighted avg       1.00      1.00      1.00       396

